In [30]:
pip install spacy-layout

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 10.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.3/178.3 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.6/148.6 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.8/80.8 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 108.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 101.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 94.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 40.1 MB/s eta 0:00:00
   ━━

In [2]:
pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 91.4 MB/s eta 0:00:00


In [6]:
import spacy
import fitz  # PyMuPDF
import re
from spacy.language import Language
from spacy.matcher import Matcher
from spacy.tokens import Span

def setup_nlp():
    nlp = spacy.load("en_core_web_sm")


    phone_matcher = Matcher(nlp.vocab)
    phone_patterns = [
        [{"TEXT": {"REGEX": r"\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}"}}],
        [{"TEXT": {"REGEX": r"\+?1?[-.\s]?\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}"}}],
        [{"TEXT": {"REGEX": r"\d{10}"}}]
    ]
    phone_matcher.add("PHONE", phone_patterns)

    # Setup address matcher
    address_matcher = Matcher(nlp.vocab)
    address_patterns = [
        [{"LIKE_NUM": True}, {"IS_ALPHA": True, "OP": "+"},
         {"TEXT": {"REGEX": r"(?i)(street|st\.?|road|rd\.?|avenue|ave\.?|boulevard|blvd\.?|drive|dr\.?|lane|ln\.?|way|court|ct\.?)"}}],
        [{"LIKE_NUM": True}, {"IS_ALPHA": True, "OP": "+"}, {"IS_ALPHA": True, "OP": "+"},
         {"TEXT": {"REGEX": r"(?i)(street|st\.?|road|rd\.?|avenue|ave\.?|boulevard|blvd\.?|drive|dr\.?|lane|ln\.?|way|court|ct\.?)"}}]
    ]
    address_matcher.add("ADDRESS", address_patterns)

    # Setup email matcher
    email_matcher = Matcher(nlp.vocab)
    email_pattern = [{"TEXT": {"REGEX": r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b"}}]
    email_matcher.add("EMAIL", [email_pattern])

    @Language.component("custom_entity_matcher")
    def custom_entity_matcher(doc):
        # Find phone numbers
        phone_matches = phone_matcher(doc)
        new_ents = []

        for match_id, start, end in phone_matches:
            span = Span(doc, start, end, label="PHONE")
            new_ents.append(span)

        # Find addresses
        address_matches = address_matcher(doc)
        for match_id, start, end in address_matches:
            span = Span(doc, start, end, label="ADDRESS")
            new_ents.append(span)

        # Find emails
        email_matches = email_matcher(doc)
        for match_id, start, end in email_matches:
            span = Span(doc, start, end, label="EMAIL")
            new_ents.append(span)

        # Combine with existing entities and remove duplicates
        all_ents = list(doc.ents) + new_ents

        # Remove overlapping entities (keep the longer ones)
        filtered_ents = []
        for ent in sorted(all_ents, key=lambda x: (x.start, -len(x.text))):
            if not any(ent.start >= existing.start and ent.end <= existing.end for existing in filtered_ents):
                filtered_ents.append(ent)

        doc.ents = filtered_ents
        return doc

    nlp.add_pipe("custom_entity_matcher", after="ner")
    return nlp

def get_entity_placeholder(label):
    """Get appropriate placeholder for entity type"""
    placeholders = {
        "PERSON": "[NAME]",
        "ORG": "[ORGANIZATION]",
        "PHONE": "[PHONE]",
        "ADDRESS": "[ADDRESS]",
        "EMAIL": "[EMAIL]",
        "GPE": "[LOCATION]",  # Geopolitical entity
        "LOC": "[LOCATION]"   # Location
    }
    return placeholders.get(label, f"[{label}]")

def anonymize_text(text, nlp):

    doc = nlp(text)

    # Sort entities by start position in reverse order to avoid index shifts
    entities = sorted(doc.ents, key=lambda x: x.start_char, reverse=True)

    anonymized_text = text
    replacements = []

    for ent in entities:
        # Skip very short entities that might be false positives
        if len(ent.text.strip()) < 2:
            continue

        placeholder = get_entity_placeholder(ent.label_)
        start_char = ent.start_char
        end_char = ent.end_char

        # Store replacement info
        replacements.append({
            'original': ent.text,
            'placeholder': placeholder,
            'start': start_char,
            'end': end_char,
            'label': ent.label_
        })

        # Replace in text
        anonymized_text = anonymized_text[:start_char] + placeholder + anonymized_text[end_char:]

    return anonymized_text, replacements

def anonymize_pdf(input_pdf_path, output_pdf_path):

    # Initialize NLP pipeline
    nlp = setup_nlp()

    # Open PDF
    pdf_document = fitz.open(input_pdf_path)

    print(f"Processing PDF with {len(pdf_document)} pages...")

    for page_num in range(len(pdf_document)):
        page = pdf_document[page_num]


        # Get text blocks with position information
        text_dict = page.get_text("dict")

        # Process each block
        for block in text_dict["blocks"]:
            if "lines" not in block:  # Skip image blocks
                continue

            for line in block["lines"]:
                for span in line["spans"]:
                    original_text = span["text"]

                    # Skip if text is too short or only whitespace
                    if len(original_text.strip()) < 2:
                        continue

                    # Anonymize the text
                    anonymized_text, replacements = anonymize_text(original_text, nlp)

                    # If replacements were made, update the PDF
                    if replacements:
                        # Get the bounding box for this text span
                        bbox = span["bbox"]
                        rect = fitz.Rect(bbox)

                        # Create a white rectangle to cover the original text
                        page.draw_rect(rect, color=(1, 1, 1), fill=(1, 1, 1))

                        # Insert the anonymized text with original formatting
                        font_size = span["size"]
                        font_name = span["font"]

                        try:
                            # Try to use the original font
                            page.insert_text(
                                (bbox[0], bbox[3]),  # Bottom-left position
                                anonymized_text,
                                fontsize=font_size,
                                fontname=font_name,
                                color=(0, 0, 0)
                            )
                        except:
                            # Fallback to default font if original font fails
                            page.insert_text(
                                (bbox[0], bbox[3]),
                                anonymized_text,
                                fontsize=font_size,
                                color=(0, 0, 0)
                            )

                        print(f"  Replaced {len(replacements)} entities in text span")
                        for repl in replacements:
                            print(f"    {repl['original']} -> {repl['placeholder']}")

    # Save the anonymized PDF
    pdf_document.save(output_pdf_path)
    pdf_document.close()
    print(f"Anonymized PDF saved to: {output_pdf_path}")

def main():
    """Main function to run the PDF anonymizer"""
    input_file = "jh.pdf"
    output_file = "jh_anonymized_report.pdf"

    try:
        anonymize_pdf(input_file, output_file)
        print("PDF anonymization completed successfully!")
    except FileNotFoundError:
        print(f"Error: Input file '{input_file}' not found.")
    except Exception as e:
        print(f"Error during PDF processing: {str(e)}")

if __name__ == "__main__":
    main()

Processing PDF with 2 pages...
  Replaced 1 entities in text span
    ANNUAL -> [DATE]
  Replaced 1 entities in text span
    FEBRUARY 2030 -> [DATE]
  Replaced 1 entities in text span
    LICERIA & CO -> [ORG]
  Replaced 1 entities in text span
    Transforming Challenges -> [NAME]
  Replaced 1 entities in text span
    2030 -> [DATE]
  Replaced 1 entities in text span
    Morgan Maxwell -> [ORG]
  Replaced 1 entities in text span
    Margarita Perez -> [NAME]
Anonymized PDF saved to: jh_anonymized_report.pdf
PDF anonymization completed successfully!
